In [2]:
import numpy as np # for arrays and math
import matplotlib.pyplot as plt # for plotting

# Conversion factors
LCGS = 1.476701332464468e+05 

In [3]:
def EOS_p2erc(p, K=100., Gamma=2):
    ene = (p/K)**(1./Gamma) + p/(Gamma-1.)
    rho = (p/K)**(1./Gamma)                       
    cs2 = K*Gamma*(Gamma-1)/(Gamma-1 + K*Gamma*rho**(Gamma-1))*rho**(Gamma-1)
    return ene, rho, cs2

def EOS_r2pe(rho, K=100., Gamma=2):
    p = K*rho**Gamma
    e = rho + p/(Gamma-1.);
    return p, e

In [4]:
def TOV(t, y):
    r = t
    m = y[0] # mass of a sphere of radius r
    p = y[1] # pressure
    ene, dummy1, dummy2 = EOS_p2erc(p) 
    dy = np.empty_like(y)
    dy[0] = 4*np.pi*ene*r**2                               
    dy[1] = -(ene+p)*(m + 4*np.pi*r**3*p)/(r*(r-2*m))
    return dy
  
def found_radius(t,y ,pfloor=0.):
  """
  Event function: Zero of pressure 
  ODE integration stops when this function returns True
  """
  return ((y[1]-pfloor)<=0.)

In [35]:
def solve_ode_euler(t, y0, dydt_fun, stop_event=None, verbose=False):
    
    N = len(t)
    dt = np.diff(t)[0]               # assume a uniformly space t array
    y = y0
    for i in range(N):
        yprev = np.copy(y)           # store previous for returning pre-event data
        y += dt * dydt_fun(t[i],y)
        if verbose:print(t[i],y)
        if stop_event: 
         if bool(stop_event(t[i],y)): 
                    print('Event reached')
                    return t[i-1], yprev 
    if stop_event: 
                    print('No event reached')
                    return t[i], y

In [36]:
rmin, rmax = 1e-6, 20. 
N = 100 # number of points between rmin and rmax
rspan = np.linspace(rmin,rmax,N)

In [37]:
rho0 = 1.28e-3 # Central (maximal) rest-mass density
p0,e0 = EOS_r2pe(rho0)
m0    = 4./3.*np.pi*e0*rmin**3
sol0 = [m0, p0]
print(sol0)


[6.047942849278782e-21, 0.00016384000000000003]


In [50]:
print("Mass and Pressure at different radius in range of rspan is given by:")
t, sol = solve_ode_euler(rspan, sol0, TOV, stop_event=found_radius, verbose=True)
print("we can see that mass approaches 1.4 solar mass and pressure drastically reduces from center to outer edge of star")

Mass and Pressure at different radius in range of rspan is given by:
1e-06 [3.66542577e-15 1.63839997e-04]
0.2020211919191919 [0.0001496 0.0001637]
0.4040413838383838 [0.0007477  0.00016314]
0.6060615757575757 [0.00209083 0.00016207]
0.8080817676767676 [0.00446993 0.0001605 ]
1.0101019595959595 [0.00816716 0.00015842]
1.2121221515151512 [0.01345275 0.00015585]
1.4141423434343432 [0.02058188 0.0001528 ]
1.6161625353535352 [0.02979174 0.00014929]
1.8181827272727271 [0.04129865 0.00014534]
2.0202029191919193 [0.05529546 0.00014099]
2.222223111111111 [0.07194903 0.00013626]
2.4242433030303028 [0.09139805 0.0001312 ]
2.626263494949495 [0.11375106 0.00012582]
2.8282836868686867 [1.39084737e-01 1.20181310e-04]
3.030303878787879 [1.67442540e-01 1.14317323e-04]
3.2323240707070706 [1.98833631e-01 1.08271592e-04]
3.4343442626262624 [2.33232149e-01 1.02087756e-04]
3.6363644545454545 [2.70576832e-01 9.58100324e-05]
3.8383846464646463 [3.10770998e-01 8.94827450e-05]
4.0404048383838385 [3.53682889e-0

In [51]:
# Get mass and radius
R = t * LCGS * 1e-5 # km 
M = sol[0] # Msun
pmin = sol[1]
print("Hence obtained minimum pressure,Radisu in Km,Mass in Solar mass are",pmin,R,M,"respectively")

Hence obtained minimum pressure,Radisu in Km,Mass in Solar mass are 2.2867788285059276e-07 12.82791140014866 1.4045085183791897 respectively
